# This is a tutorial for automatically querying chatgpt for code
> If you need help you can reach me (the creator of this code) at khughes@nyu.edu

In [1]:
import os
import openai
# Note: if you do not have this package please install it with your package manager of choice
# Example: https://pypi.org/project/openai/

## Step 1) set your API key

In [6]:
"""
If you dont have your api key you can generate one for free by following this tutorial here:
https://gptforwork.com/help/gpt-for-docs/setup/create-openai-api-key

For saftey, you can also set it up as an environmental variable via this tutorial:
https://platform.openai.com/docs/quickstart?context=python
"""
openai.api_key = os.environ['OPENAI_API_KEY']

## Step 2) Define your gpt call function

In [7]:
def run_gpt4(messages, text_only=True, this_model="gpt-4"):
    """ main function that query's chatgpt through openai's api """
    response = openai.ChatCompletion.create(
        model=this_model,
        messages=messages
    )
    if text_only:  # main response you care about
        return response["choices"][0]["message"]["content"]
    else:
        return response  # entire response structure

## Step 3) Use call function in some context

In [ ]:
# context 1: you just wana take input 1 by 1
def chat_with_gpt(previous_hist=[]):
    """ continuous conversation with chatgpt given user input """
    message_hist = []  # init
    if len(previous_hist) == 0:
        message_hist = [{"role": "system", "content": "You are a helpful assistant."}]
    else:
        message_hist = previous_hist
       
    user_message = input("user: ")
    while True:
        try:
            if user_message == "exit":
                return message_hist
            elif user_message[0] == "\\":  # add context to chat gpt with a backslash
                message_hist.append({"role": "system", "content": user_message})
                print("chat_gpt: ", user_message, '\n')
            else:
                message_hist.append({"role": "user", "content": user_message})
                response = run_gpt4(message_hist)
                print("chat_gpt: ", response, '\n')
                message_hist.append({"role": "system", "content": response})
        except:  # Note: sometimes it will crash cuz the api is inconsistent, just try again
            print("    ~encountered api error, retrying...")
        user_message = input("user: ")

print("\n\n", chat_with_gpt())

user: I'm a confused student, can you help me with some code?
chat_gpt:  Of course, I'd be happy to help you with your code. Could you please provide me with more details about the specific problem or issue you're facing? 

user: I'm trying to make a bot to automatically query chatgpt
chat_gpt:  In order to create a bot that interacts with the ChatGPT API, you'll have to use some sort of back-end language. Let's assume we're using Python. You would first need to install the OpenAI GPT-3 API using pip:

```bash
pip install openai
```

Then you could use the following Python code to make your query:

```python
import openai

openai.api_key = 'your-api-key'

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ]
)

print(response['choices'][0]['message']['content'])
```

Note that you need to replace `'your-api-key'`

In [49]:
# context 2: you want to 1shot a query for automation
def single_query(input_prompt, directive="You are a helpful bot that assists students with their code.", verbose=False):
    """ ask a single query to chat gpt given an input directive 
    Note: feel free to change the directive
    """
    message_hist = [{"role": "system", "content": directive},
                    {"role": "user", "content": input_prompt}]
    response = run_gpt4(message_hist)
    if verbose:
        print("chat_gpt: ", response, '\n')
    message_hist.append({"role": "system", "content": response})
    return response


examples = ["give me a piece of python code that returns even numbers from 1 to i please.",
            "give me a piece of python code that returns n numbers in the fibbonacci sequence please."]
answers = [single_query(question, '') for question in examples]  # running queries

for i in range(len(answers)): # showing queries
    print(examples[i], '\n', answers[i])
    print('\n')

give me a piece of python code that returns even numbers from 1 to i please. 
 Sure, here's a simple Python function that will return a list of even numbers from 1 to i:

```python
def even_numbers(i):
    return [num for num in range(1, i + 1) if num % 2 == 0]
```

To use the function, just call it with the number i you want:
```python
even_numbers(10)
```

This will return a list of even numbers from 1 to 10.


give me a piece of python code that returns n numbers in the fibbonacci sequence please. 
 Sure, here is a simple Python function to generate 'n' numbers in the Fibonacci sequence:

```python
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

print(fibonacci(10))  # it will print: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
```

The `fibonacci` function begins with a list containing the first two numbers of the Fibonacci sequence. It then enters a loop, which continues

## Step 4) Parse Code:

In [47]:
coded_answers = []
# for each answer, get a list of code chunks gpt returned
for ans in answers:
    print("For Question", (len(coded_answers)+1), "-->")
    this_code = []
    for i, code_chunk in enumerate(ans.split('```')):  # pieces of code are denoted by ``` so we split
        if i%2 == 1:  # every other chunk is a piece of code in this case
            this_code.append(code_chunk[7:])  # Note: code declarations also have 'python\n' denoting the language, since we dont need this, we omit the first 7 chars
            print("\nCode Found:")
            print(code_chunk[7:])
    print('\n')
    coded_answers.append(this_code)

# can do further cleaning like deleting blank lines or comments

For Question 1 -->

Code Found:
def even_numbers(i):
    return [num for num in range(1, i + 1) if num % 2 == 0]


Code Found:
even_numbers(10)



For Question 2 -->

Code Found:
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

print(fibonacci(10))  # it will print: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]





## Step 4: Run Parsed Code

In [51]:
for i, ans in enumerate(coded_answers):
    print("\nExecuting Answer #"+str(i+1), "code...")
    exec('\n'.join(ans))  # join code blocks to execute them all at once
# Note: exec() is python specific


Executing Answer #1 code...

Executing Answer #2 code...
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


> Food for thought; how can we properly evaluate these? what exceptions should we handle? how does the coding language affect the output? what other contexts can this be useful for?